In [6]:
#importing the classes and functions
from math import sqrt
import numpy as np
from numpy import array
import pyspark
from pyspark import *
from pyspark.conf import *
from pyspark.sql import *
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.clustering import KMeans

In [2]:
#setting the string variables
app_name = 'Item-User Matrix'
master = 'local'

#setting the data file for item-user matrix and num of clusters
itemusermatdata_path = 'itemusermat.data'
num_cluster = 10

#setting the data file for movies
moviesdata_path = 'movies.data'

In [3]:
#configuring the Spark and setting the master & app name
spark = SparkConf().setAppName(app_name).setMaster(master)
sc = SparkContext(conf=spark)

In [8]:
def movies_line_mapper(line):
    data = line.split('::')
    data[2] = data[2].split('|')
    genre=''
    for i in data[2]:
        genre += i + ', '
    genre = genre[:-2]
    data[2] = genre
    return data

def itemuser_mat_mapper(line):
    data = line.split(' ')
    for i in data:
        i=float(i)
    return Vectors.dense(data)

In [14]:
# Load and parse the data --> Item-User Matrix Data
itemuser_data_rdd = sc.textFile(itemusermatdata_path)
parsed_itemuser_data = itemuser_data_rdd.map(itemuser_mat_mapper)

data=parsed_itemuser_data.collect()
for i in range(5):
    print(data[i])
    
# Load and parse the data --> Movies Data
moviesdata_rdd = sc.textFile(moviesdata_path)
parsed_movies_data = moviesdata_rdd.map(movies_line_mapper)

data = parsed_movies_data.collect()
for i in range(5):
    print(data[i])

[1718.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
# Build the model (cluster the data)
kmeansModel = KMeans.train(parsed_itemuser_data, num_cluster,maxIterations=100)

#get cluster number for each data data point
predicted_data = kmeansModel.predict(parsed_itemuser_data)

In [ ]:
def modify_zipped_rdd_mapper(line):
    temp = np.append(line[0],line[1])
    return temp

In [ ]:
# data = predicted_data.collect()
# # print(data)

zipped_cluster_itemuser = parsed_itemuser_data.zip(predicted_data)
zipped_rdd = zipped_cluster_itemuser.map(modify_zipped_rdd_mapper)
print(zipped_rdd)

# data = zipped_rdd.collect()
# for i in range(5):
#     dummy = np.append(data[i][0],data[i][1])
#     print(dummy)